In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import IntegerType

In [2]:
from pyspark.sql.functions import *

In [10]:
spark = SparkSession.builder.master("local[2]").appName("DataSet").getOrCreate()

In [11]:
data = [
    {"fname": "Rahul", "lname": "Kumar", "gender": "M", "salary": 10000, "bonus": 1000, "deduction": 500},
    {"fname": "Taukir", "lname": "Khan", "gender": "M", "salary": 12000, "bonus": 2000},
    {"fname": "Pooja", "lname": "Sharma", "gender": "F", "salary": 15000, "bonus": 3000, "deduction": 2000},
     {"fname": "Ravi", "lname": "Kumari", "gender": "other", "salary": 15000, "bonus": 3000, "deduction": 2000}
]

#### Conver data to blow format
```
 +------------+------+------------+
|        name|gender|total_salary|
+------------+------+------------+
| Rahul Kumar|  Male|       10500|
| Taukir Khan|  Male|       14000|
|Pooja Sharma|Female|       16000|
| Ravi Kumari| other|       16000|
+------------+------+------------+

```

In [4]:
spark = SparkSession.builder.master("local[2]").appName("DataSet").getOrCreate()

In [5]:
dataframe = spark.createDataFrame(data) 
dataframe.show()
# na.fill(value=0,subset=["deduction"]).

+-----+---------+------+------+------+------+
|bonus|deduction| fname|gender| lname|salary|
+-----+---------+------+------+------+------+
| 1000|      500| Rahul|     M| Kumar| 10000|
| 2000|     NULL|Taukir|     M|  Khan| 12000|
| 3000|     2000| Pooja|     F|Sharma| 15000|
| 3000|     2000|  Ravi| other|Kumari| 15000|
+-----+---------+------+------+------+------+



#### Method-1

In [9]:
dataframe.na.fill(value=0,subset=["deduction"]).withColumn("gender",functions.when(dataframe.gender == "M","Male").\
                                  when(dataframe.gender == "F","Female").\
                                  otherwise(dataframe.gender)).\
withColumn("name", functions.expr("concat(fname,' ',lname)")).\
withColumn("total_salary",(functions.col('salary') + functions.col('bonus') - functions.col('deduction'))).\
select("name","gender","total_salary").show()

+------------+------+------------+
|        name|gender|total_salary|
+------------+------+------------+
| Rahul Kumar|  Male|       10500|
| Taukir Khan|  Male|       14000|
|Pooja Sharma|Female|       16000|
| Ravi Kumari| other|       16000|
+------------+------+------------+



#### Method-2

In [7]:
dataframe.na.fill(value=0,subset=["deduction"]).withColumn("gender",functions.expr("CASE WHEN gender = 'M' THEN 'Male' " +
           "WHEN gender = 'F' THEN 'Female' ELSE gender END")).\
withColumn("name", functions.expr("concat(fname,' ',lname)")).\
withColumn("total_salary",(functions.col('salary') + functions.col('bonus') - functions.col('deduction'))).\
select("name","gender","total_salary").show()

+------------+------+------------+
|        name|gender|total_salary|
+------------+------+------------+
| Rahul Kumar|  Male|       10500|
| Taukir Khan|  Male|       14000|
|Pooja Sharma|Female|       16000|
| Ravi Kumari| other|       16000|
+------------+------+------------+



#### Method-3

In [8]:
dataframe.withColumn("gender",functions.expr("CASE WHEN gender = 'M' THEN 'Male' " +
           "WHEN gender = 'F' THEN 'Female' ELSE gender END")).\
withColumn("name", functions.expr("concat(fname,' ',lname)")).\
withColumn("total_salary",(col('salary') + col('bonus') - coalesce(col('deduction'),lit(0.0)))).\
select("name","gender","total_salary").show()

+------------+------+------------+
|        name|gender|total_salary|
+------------+------+------------+
| Rahul Kumar|  Male|     10500.0|
| Taukir Khan|  Male|     14000.0|
|Pooja Sharma|Female|     16000.0|
| Ravi Kumari| other|     16000.0|
+------------+------+------------+

